# word2vec: How To Prep Word Vectors For Modeling

### Train Our Own Model

In [1]:
# Read in the data, clean it, split it into train and test sets, and then train a word2vec model
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]


messages['text_clean'] = messages['text'].apply(lambda x: gensim.utils.simple_preprocess(x))
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

w2v_model = gensim.models.Word2Vec(X_train,
                                   size=100,
                                   window=5,
                                   min_count=2)

### Prep Word Vectors

In [2]:
# Generate a list of words the word2vec model learned word vectors for
w2v_model.wv.index2word

['you',
 'to',
 'the',
 'and',
 'in',
 'is',
 'me',
 'it',
 'my',
 'for',
 'your',
 'call',
 'of',
 'that',
 'have',
 'on',
 'now',
 'are',
 'can',
 'so',
 'but',
 'not',
 'or',
 'we',
 'do',
 'get',
 'at',
 'will',
 'no',
 'ur',
 'if',
 'be',
 'with',
 'just',
 'this',
 'when',
 'gt',
 'how',
 'lt',
 'ok',
 'up',
 'from',
 'go',
 'll',
 'out',
 'free',
 'what',
 'all',
 'know',
 'like',
 'got',
 'am',
 'then',
 'come',
 'was',
 'there',
 'good',
 'he',
 'day',
 'its',
 'time',
 'only',
 'love',
 'text',
 'send',
 'one',
 'txt',
 'want',
 'as',
 'she',
 'home',
 'by',
 'need',
 'going',
 'still',
 'about',
 'see',
 'don',
 'stop',
 'today',
 'back',
 'lor',
 'sorry',
 'tell',
 'da',
 'new',
 'later',
 'reply',
 'take',
 'any',
 'mobile',
 'pls',
 'please',
 'her',
 'they',
 'ì_',
 'dear',
 'our',
 'dont',
 'been',
 'phone',
 'some',
 'think',
 'hi',
 'did',
 'here',
 'week',
 'where',
 'more',
 're',
 'hope',
 'well',
 'who',
 'night',
 'has',
 'much',
 'oh',
 'him',
 'an',
 'too',
 'w

In [3]:
# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
w2v_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_model.wv.index2word])
                     for ls in X_test])

/home/cherry/documents/projects/learnDataScience/machineLearning/Adv_NLP_Python_ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Why is the length of the sentence different than the length of the sentence vector?
for i,v in enumerate(w2v_vect):
    print(len(X_test.iloc[i]),len(v))

7 7
14 14
20 13
14 14
30 29
9 8
15 15
8 8
26 24
25 20
13 13
25 22
10 10
5 5
5 3
17 14
4 4
26 26
9 9
8 8
6 6
21 21
6 4
41 34
5 5
12 11
24 23
31 30
50 45
22 20
21 21
6 6
7 6
8 7
41 40
12 10
14 12
22 19
28 28
10 10
8 8
25 19
12 11
6 5
11 10
7 7
5 5
21 19
18 18
7 7
11 11
21 20
5 5
7 7
12 11
9 9
37 28
6 5
17 16
24 24
6 6
2 2
9 9
7 7
21 20
12 12
6 6
19 19
12 8
21 21
29 29
11 10
8 6
21 21
29 26
2 2
31 31
9 7
6 6
25 25
30 25
10 8
14 13
10 4
7 5
7 7
14 14
12 12
26 25
4 3
1 1
29 28
5 5
11 5
7 6
16 15
5 5
9 5
5 5
36 34
9 6
8 8
8 8
12 12
5 3
22 17
3 3
9 8
17 16
10 10
3 1
27 27
5 5
2 2
6 6
27 25
1 1
8 8
8 7
28 21
24 22
8 6
16 15
26 25
7 7
9 9
67 66
23 18
6 6
5 5
4 4
11 9
6 6
9 9
15 13
6 6
23 23
12 10
5 3
14 13
7 6
29 28
21 19
18 18
8 8
5 4
46 43
24 22
10 8
25 23
1 1
11 8
21 21
13 13
17 15
14 14
10 10
12 12
4 4
20 19
7 7
5 5
7 7
10 8
24 21
25 24
24 22
7 7
9 9
6 6
5 5
6 6
11 10
30 28
2 2
27 27
1 0
21 21
6 6
8 8
7 7
5 5
8 6
4 4
23 20
5 4
15 14
13 12
21 19
10 9
8 8
13 13
9 8
9 9
7 7
1 1
8 6
29 24
31 31

In [7]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
w2v_vect_avg = []

for vect in w2v_vect:
    if len(vect)!=0:
        w2v_vect_avg.append(vect.mean(axis=0))
    else:
        w2v_vect_avg.append(np.zeros(100))

In [6]:
# Are our sentence vector lengths consistent?
for i,v in enumerate(w2v_vect_avg):
    print(len(X_test.iloc[i]),len(v))

7 100
14 100
20 100
14 100
30 100
9 100
15 100
8 100
26 100
25 100
13 100
25 100
10 100
5 100
5 100
17 100
4 100
26 100
9 100
8 100
6 100
21 100
6 100
41 100
5 100
12 100
24 100
31 100
50 100
22 100
21 100
6 100
7 100
8 100
41 100
12 100
14 100
22 100
28 100
10 100
8 100
25 100
12 100
6 100
11 100
7 100
5 100
21 100
18 100
7 100
11 100
21 100
5 100
7 100
12 100
9 100
37 100
6 100
17 100
24 100
6 100
2 100
9 100
7 100
21 100
12 100
6 100
19 100
12 100
21 100
29 100
11 100
8 100
21 100
29 100
2 100
31 100
9 100
6 100
25 100
30 100
10 100
14 100
10 100
7 100
7 100
14 100
12 100
26 100
4 100
1 100
29 100
5 100
11 100
7 100
16 100
5 100
9 100
5 100
36 100
9 100
8 100
8 100
12 100
5 100
22 100
3 100
9 100
17 100
10 100
3 100
27 100
5 100
2 100
6 100
27 100
1 100
8 100
8 100
28 100
24 100
8 100
16 100
26 100
7 100
9 100
67 100
23 100
6 100
5 100
4 100
11 100
6 100
9 100
15 100
6 100
23 100
12 100
5 100
14 100
7 100
29 100
21 100
18 100
8 100
5 100
46 100
24 100
10 100
25 100
1 100
11 100
21 1